In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
import os

In [8]:
img_size = 784
hidden_dim = 400
latent_dim = 20
batch_size = 128
epochs = 10

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
train_dataset = torchvision.datasets.FashionMNIST(root="./data",
                                                  train = True,
                                                  transform = transforms.ToTensor(),
                                                  download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_dataset = torchvision.datasets.FashionMNIST(root="./data",
                                                  train = False,
                                                  transform = transforms.ToTensor(),
                                                  download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

# save img to dir
sample_dir = "imgs"

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [11]:
# Model

class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()

        self.fc1 = nn.Linear(img_size,hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim,latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim,latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, img_size)

    def encode(self, x):
        h = self.F.relu(self.fc1(x))
        mu = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)

        return mu, logvar

    def reparameterize(self, mu,logvar):
        std = torch.exp(logvar/2)
        eps = torch.randn_like(std)
        
        return mu + eps*std
        
    def decode(self,z):
        h = F.relu(self.fc3(z))
        out = torch.sigmoid(self.fc4(h))

        return out

    def forward(x):
        mu,logvar = self.encode(x.view(-1,img_size))
        z = self.reparameterize(mu,logv)
        out = self.decode(z)

        return out,mu,logvar


model = VAE().to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
def loss_fn(out,img,mu,logvar):
    bce = F.binary_cross_entropy(out,img.view(-1,img_size), reduction="sum")
    kld = 0.5*torch.sum(logvar.exp() + mu**2 - 1 - logvar)

    return bce+kld

def train(epoch):
    model.train()
    train_loss = 0

    for i,(images,_) in enumerate(train_loader):
        images = images.to(device)
        out,mu,logvar = model(images)
        loss = loss_fn(out,images,mu,logvar)
        optim.zero_grad()
        loss.backward()
        optim.step()

        train_loss+=loss.item()

        if i%100==0:
            print("Train Epoch {} [Batch {}/{}]\tLoss: {:3f}".format(epoch, i, len(train_loader), loss.item()/len(images))
    
    print(f"Epoch {epoch}, Average Loss: {(train_loss/len(train_loader.dataset)):.3f}")
                  
def test(epoch):
    model.eval()

    test_loss = 0

    with torch.no_grad():
        for batch_idx,(images,_) in enumerate(test_loader):
            images = images.to(device)
            out,mu,logvar = model(images)
            test_loss += loss_fn(out,images,mu,logvar).item()

            if batch_idx == 0:
                comparison = torch.cat([images[:5], out.view(batch_size,1,28,28)[:5]])
                save_image(comparison.cpu(), 'results/out_' + str(epoch) + '.png',nrow=5)
                
        print(">> Average Loss: {:..3f}".format(test_loss/len(test_loader.dataset)))


SyntaxError: invalid syntax (378109805.py, line 24)

In [ ]:
len(train_loader.dataset)

In [ ]:
for epoch in range(1,epoch+1):
    train(epoch)
    test(epoch)

    with torch.no_grad:
        sample = torch.randn(64,20).to(device)
        generated = model.decode(sample).cpu()
        save_img(generated.view(64,1,28,28), 'results/sample_' + str(epoch)+'.png')